# Mammary Adenoma 추출 PoC (v1)

이 노트북은 앞서 설명한 단계별 필터링 절차를 적용하여 `Data/조직검사 결과 매칭(2024)_utf8_pruned.csv`에서 확실한 **Mammary Adenoma** 증례를 추출합니다.

<sub>규칙 기반 1차 필터와 수동 검토 루프를 묶어 약한 감독(weak supervision) 방식으로 레이블링 비용을 낮추는 PoC 흐름을 검증합니다.</sub>

In [ ]:
import pandas as pd
from pathlib import Path

pd.options.display.max_colwidth = 120

DATA_PATH = Path("../Data/조직검사 결과 매칭(2024)_utf8_pruned.csv")
assert DATA_PATH.exists(), f"Missing data file: {DATA_PATH}"

raw_df = pd.read_csv(DATA_PATH)
raw_df.head()

## 1. 스키마와 초기 건수 확인

<sub>스키마와 총 건수를 미리 확인해야 이후 필터링에서 열 이름 오타나 누락으로 인한 정보 손실을 방지할 수 있습니다.</sub>

In [ ]:
raw_df.info()

## 2. 텍스트 정규화 도우미

소문자 변환, HTML `<br>` 제거, 공백 정리, 그리고 느슨한 매칭을 위한 알파벳 전용 보조 문자열을 생성합니다.

<sub>서술문이 길거나 HTML 줄바꿈이 섞인 경우도 일관된 형태로 맞추기 위해 정규화를 먼저 수행합니다 (예: `Mammary Adenoma<br>complete excision` → `mammary adenoma complete excision`).</sub>

In [ ]:
import re

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = s.replace('<br>', ' ')
    s = re.sub(r"[^a-z0-9\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def alpha_only(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-z\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

norm_cols = ['DIAGNOSIS', 'GROSS_FINDINGS', 'COMMENTS', 'MICROSCOPIC_FINDINGS']
norm_df = raw_df.copy()
for col in norm_cols:
    norm_df[f"{col}_norm"] = norm_df[col].apply(normalize_text)
    norm_df[f"{col}_alpha"] = norm_df[col].apply(alpha_only)

norm_df[[c for c in norm_df.columns if c.endswith('_norm')]].head()

## 3. 키워드 규칙

확정 매칭용 기본 패턴과 다른 컬럼의 보조 컨텍스트 용어를 정의합니다.

<sub>자유 서술식으로 입력된 진단명(`Apocrine or mammary adenoma, completely excised` 등)을 포착하기 위해 강한 패턴과 약한 보조 컨텍스트를 분리해 관리합니다.</sub>

In [ ]:
primary_patterns = [
    r"mammary adenoma",
    r"mamary adenoma",  # common misspelling
    r"mammary benign adenoma",
    r"apocrine or mammary adenoma",
    r"mammary adenom[ae]",
]

support_keywords = {
    'COMMENTS': ["adenoma", "유선", "apocrine", "benign", "adenomatous"],
    'MICROSCOPIC_FINDINGS': ["adenoma", "lobule", "capsulated", "ductal", "benign"],
    'GROSS_FINDINGS': ["mammary", "乳腺", "mamary", "mass", "nodule"],
}

# loose requirement: both tokens present somewhere in diagnosis alpha-only
loose_terms = ["mammary", "mamary"]


## 4. 점수 산정 함수

- 확정 진단 패턴 매칭 시 +2
- 알파벳 전용 문자열에서 'mammary/mamary'와 'adenoma'가 함께 등장하면 +1
- 지원 컬럼에 보조 키워드가 있으면 컬럼당 +1
- 최종 라벨: `score >= 2`이면 확실한 Mammary Adenoma

<sub>점수 기반으로 묶으면 새 키워드를 추가하거나 임계값을 조정할 때 재현율/정밀도를 유연하게 맞출 수 있고, 경계 사례를 따로 검토하는 근거가 됩니다.</sub>

In [ ]:
def score_row(row):
    score = 0
    diag_norm = row['DIAGNOSIS_norm']
    diag_alpha = row['DIAGNOSIS_alpha']

    # primary patterns
    if any(re.search(pat, diag_norm) for pat in primary_patterns):
        score += 2

    # loose co-occurrence
    if all(term in diag_alpha.split() for term in loose_terms) and 'adenoma' in diag_alpha:
        score += 1

    # supporting columns
    for col, keywords in support_keywords.items():
        text = row[f"{col}_norm"]
        if any(kw in text for kw in keywords):
            score += 1
    return score

norm_df['score'] = norm_df.apply(score_row, axis=1)
norm_df['is_mammary_adenoma'] = norm_df['score'] >= 2

norm_df[['DIAGNOSIS', 'score', 'is_mammary_adenoma']].head()

## 5. 경계 사례 검토

임계값 근처의 예시를 보여 수동 검토에 활용합니다.

<sub>예를 들어 `DIAGNOSIS`에는 'cyst'만 있으나 `COMMENTS`에서 'mammary adenoma favored'라고 언급된 행처럼 혼합 표현을 사람이 빨리 확인할 수 있습니다.</sub>

In [ ]:
borderline = norm_df[norm_df['score'] == 1]
borderline[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'score']].head(10)

## 6. 확실 양성 사례 스냅샷

<sub>점수 규칙이 제대로 작동하는지, 해부 부위나 진단 요약이 기대대로 나오는지 빠르게 눈으로 검증합니다.</sub>

In [ ]:
positives = norm_df[norm_df['is_mammary_adenoma']]
positives[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score']].head(10)

## 7. 요약 통계

<sub>필터 적용 전후 건수, 확정/경계 비율을 보면 규칙 조정이 필요한지 판단할 수 있습니다.</sub>

In [ ]:
total = len(norm_df)
positives_count = positives.shape[0]
print(f"Total rows: {total}")
print(f"Confident Mammary Adenoma: {positives_count} ({positives_count/total:.2%})")
score_counts = norm_df['score'].value_counts().sort_index()
score_counts

## 8. 정제된 데이터셋 저장

<sub>후속 분석·모델 학습에서 동일한 결과를 재사용할 수 있도록 버전이 명시된 CSV를 남깁니다.</sub>

In [ ]:
output_cols = [
    'INSP_RQST_NO', 'FOLDER', 'FILE_NAME', 'SITE',
    'DIAGNOSIS', 'GROSS_FINDINGS', 'COMMENTS', 'MICROSCOPIC_FINDINGS',
    'score', 'is_mammary_adenoma'
]

curated = norm_df[output_cols].copy()
output_path = Path("./mammary_adenoma_curated.csv")
curated.to_csv(output_path, index=False)
output_path

## 9. 다음 단계

- 경계 사례 수동 검토 후 keyword 리스트를 보완합니다.
- 희귀 동의어에 대한 recall을 높이려면 TF-IDF나 임베딩 유사도 방식을 고려합니다.

<sub>규칙→수동 리뷰→키워드 보완→모델 전환의 반복 학습 루프를 염두에 둔 로드맵입니다.</sub>